In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_name = "Qwen/Qwen3-8B"

In [ ]:
# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

In [ ]:
!nvidia-smi

In [ ]:
# prepare the model input
# prompt = "Explain O'Reilly online learning"
# prompt = "How many 'i's are in 'inscription'?"
prompt = "How many 'r's are in 'strawberry'?"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)

In [ ]:
# we check what happens when we remove thinking later
text

In [ ]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

In [ ]:
%%time 
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)

In [ ]:
# only read output, skip input
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

In [ ]:
len(output_ids)

In [ ]:
# possibly two different content parts
try:
    # rindex finding 151668 (</think>), could also look for the string!
    think_index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    think_index = 0

thinking_content = tokenizer.decode(output_ids[:think_index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[think_index:], skip_special_tokens=True).strip("\n")

In [ ]:
print(thinking_content)

In [ ]:
print(content)

In [ ]:
from IPython.display import display, Markdown
display(Markdown(content))